In [144]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import os
import sys
import glob
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
pd.set_option('display.max_columns', None)

In [44]:
players = pd.read_csv('/home/ryan/ml/tennis_ml/tennis_atp/atp_players.csv', 
                      header=None, 
                      names=['player_id', 'first_name', 'last_name', 'hand', 'birthday','country_code'])

In [45]:
players

,player_id,first_name,last_name,hand,birthday,country_code
0,100001,Gardnar,Mulloy,R,19131122.0,USA
1,100002,Pancho,Segura,R,19210620.0,ECU
2,100003,Frank,Sedgman,R,19271002.0,AUS
3,100004,Giuseppe,Merlo,R,19271011.0,ITA
4,100005,Richard Pancho,Gonzales,R,19280509.0,USA
...,...,...,...,...,...,...
54892,209858,Vladimir,Osminkin,U,NaN,RUS
54893,209859,Rostislav,Galfinger,U,20021112.0,KAZ
54894,209860,Ignacio,Buse,U,20040325.0,PER
54895,209861,Maj,Tomac,U,20010504.0,SLO


In [48]:
df = pd.read_csv('/home/ryan/ml/tennis_ml/tennis_atp/atp_matches_2019.csv')

In [49]:
list(df.columns.values)

['tourney_id',
 'tourney_name',
 'surface',
 'draw_size',
 'tourney_level',
 'tourney_date',
 'match_num',
 'winner_id',
 'winner_seed',
 'winner_entry',
 'winner_name',
 'winner_hand',
 'winner_ht',
 'winner_ioc',
 'winner_age',
 'loser_id',
 'loser_seed',
 'loser_entry',
 'loser_name',
 'loser_hand',
 'loser_ht',
 'loser_ioc',
 'loser_age',
 'score',
 'best_of',
 'round',
 'minutes',
 'w_ace',
 'w_df',
 'w_svpt',
 'w_1stIn',
 'w_1stWon',
 'w_2ndWon',
 'w_SvGms',
 'w_bpSaved',
 'w_bpFaced',
 'l_ace',
 'l_df',
 'l_svpt',
 'l_1stIn',
 'l_1stWon',
 'l_2ndWon',
 'l_SvGms',
 'l_bpSaved',
 'l_bpFaced',
 'winner_rank',
 'winner_rank_points',
 'loser_rank',
 'loser_rank_points']

In [207]:
#iterate through player ids and collect features
target_pid = 105453
df2 = df[(df.loc[:,'winner_id'] == target_pid ) | (df.loc[:,'loser_id'] == target_pid)]

#create target column here...
y = [1 if pid==target_pid else 0 for pid in df2.winner_id.values]
y = pd.DataFrame(data=y, columns=['target'])

#switch up the dataframe to account for losses since after 
#taking target into account, we dont care about winner/loser columns
y['target']

0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     0
9     0
10    1
11    1
12    1
13    0
14    1
15    0
16    1
17    0
18    0
19    0
20    1
21    1
22    1
23    0
24    1
25    0
26    1
27    1
28    1
29    1
30    1
31    1
32    0
33    1
34    1
35    1
36    1
37    0
38    0
39    0
40    1
41    1
42    0
Name: target, dtype: int64

In [209]:
idx = (y['target'] == 1)
df2.loc[idx,['w_ace','l_ace']] = df2.loc[idx,['l_ace','w_ace']].values

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [169]:
df2.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2019-M020,Brisbane,Hard,32,A,20181231,300,105453,2,NaN,Kei Nishikori,R,178.0,JPN,29.004791,106421,4,NaN,Daniil Medvedev,R,NaN,RUS,22.885695,6-4 3-6 6-2,3,F,124.0,3.0,3.0,77.0,44.0,31.0,17.0,13.0,3.0,6.0,8.0,6.0,100.0,54.0,34.0,20.0,14.0,10.0,15.0,9.0,3590.0,16.0,1977.0
2,2019-M020,Brisbane,Hard,32,A,20181231,298,105453,2,NaN,Kei Nishikori,R,178.0,JPN,29.004791,104871,NaN,NaN,Jeremy Chardy,R,188.0,FRA,31.882272,6-2 6-2,3,SF,66.0,2.0,2.0,47.0,33.0,26.0,9.0,8.0,2.0,2.0,10.0,3.0,46.0,27.0,15.0,6.0,8.0,1.0,5.0,9.0,3590.0,40.0,1050.0
6,2019-M020,Brisbane,Hard,32,A,20181231,294,105453,2,NaN,Kei Nishikori,R,178.0,JPN,29.004791,105777,6,NaN,Grigor Dimitrov,R,188.0,BUL,27.627652,7-5 7-5,3,QF,100.0,5.0,2.0,59.0,47.0,37.0,9.0,12.0,0.0,1.0,8.0,5.0,80.0,53.0,37.0,13.0,12.0,6.0,9.0,9.0,3590.0,19.0,1835.0
14,2019-M020,Brisbane,Hard,32,A,20181231,286,105453,2,NaN,Kei Nishikori,R,178.0,JPN,29.004791,106045,NaN,NaN,Denis Kudla,R,180.0,USA,26.370979,7-5 6-2,3,R16,79.0,7.0,2.0,56.0,32.0,27.0,15.0,10.0,0.0,0.0,5.0,3.0,60.0,36.0,26.0,8.0,10.0,4.0,7.0,9.0,3590.0,63.0,810.0
154,2019-580,Australian Open,Hard,128,G,20190114,115,105453,8,NaN,Kei Nishikori,R,178.0,JPN,29.043121,111794,Q,NaN,Kamil Majchrzak,R,NaN,POL,23.003422,3-6 6-7(6) 6-0 6-2 3-0 RET,5,R128,168.0,6.0,3.0,109.0,73.0,53.0,20.0,19.0,6.0,8.0,11.0,9.0,123.0,65.0,42.0,22.0,19.0,6.0,14.0,9.0,3750.0,176.0,291.0


In [ ]:
#sort by tournament date to feature are sequential/time-dependant
df2 = df2.sort_values('tourney_date')
df2

In [ ]:
plt.figure(figsize=(15,7))
plt.plot(df2.w_ace.values, color='b')
plt.plot(df2.w_df.values, color='r')
plt.plot(df2.w_bpFaced.values, color='g')
plt.plot(df2.w_bpSaved.values, 'g--')
plt.show()

In [193]:
segments = df2.shape[0]

#temp features list
features = ['w_ace', 'w_bpFaced']

x_tr = extract_features(df2, segments, features)

/home/ryan/.conda/envs/tf-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:55: RuntimeWarning: Mean of empty slice.
/home/ryan/.conda/envs/tf-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:60: RuntimeWarning: Mean of empty slice.
/home/ryan/.conda/envs/tf-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: divide by zero encountered in true_divide
/home/ryan/.conda/envs/tf-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in true_divide


In [194]:
x_tr

,w_ace_mean,w_ace_std,w_ace_max,w_ace_min,w_ace_mean_change_abs,w_ace_abs_max,w_ace_abs_min,ave_roll_std_2,std_roll_std_2,av_change_abs_roll_std_2,av_change_rate_roll_std_2,ave_roll_mean_2,std_roll_mean_2,av_change_abs_roll_mean_2,av_change_rate_roll_mean_2,ave_roll_std_4,std_roll_std_4,av_change_abs_roll_std_4,av_change_rate_roll_std_4,ave_roll_mean_4,std_roll_mean_4,av_change_abs_roll_mean_4,av_change_rate_roll_mean_4,ave_roll_std_6,std_roll_std_6,av_change_abs_roll_std_6,av_change_rate_roll_std_6,ave_roll_mean_6,std_roll_mean_6,av_change_abs_roll_mean_6,av_change_rate_roll_mean_6,w_bpFaced_mean,w_bpFaced_std,w_bpFaced_max,w_bpFaced_min,w_bpFaced_mean_change_abs,w_bpFaced_abs_max,w_bpFaced_abs_min
0,3.000000,0.000000,3.0,3.0,0.000000,3.0,3.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,0.000000,6.0,6.0,0.000000,6.0,6.0
1,2.500000,0.707107,3.0,2.0,-1.000000,3.0,2.0,2.828427,0.000000,0.000000e+00,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2.828427,6.0,2.0,-4.000000,6.0,2.0
2,3.333333,1.527525,5.0,2.0,1.000000,5.0,2.0,1.767767,1.060660,-2.121320e+00,0.000000,2.750000,1.250000,-2.500000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,2.645751,6.0,1.0,-2.500000,6.0,1.0
3,4.250000,2.217356,7.0,2.0,1.333333,7.0,2.0,1.414214,1.000000,-1.060660e+00,0.000000,2.000000,1.471960,-1.750000,0.500000,2.629956,0.000000,0.000000,0.0,2.250000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.250000,2.629956,6.0,0.0,-2.000000,6.0,0.0
4,4.600000,2.073644,7.0,2.0,0.750000,7.0,2.0,2.474874,2.031010,9.428090e-01,1.000000,2.500000,1.541104,0.000000,1.000000,3.111966,0.482010,0.964021,0.0,2.500000,0.250000,0.500000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.400000,3.435113,8.0,0.0,0.500000,8.0,0.0
5,5.333333,2.581989,9.0,2.0,1.200000,9.0,2.0,2.404163,1.822087,-1.767767e-01,1.666667,3.300000,2.111871,0.625000,1.500000,3.306926,0.480529,0.533445,0.5,2.833333,0.513701,0.625000,0.5,3.141125,0.000000,0.000000,0.000000,3.666667,0.000000,0.000000,0.000000,3.666667,3.141125,8.0,0.0,-0.200000,8.0,0.0
6,5.000000,2.516611,9.0,2.0,0.000000,9.0,2.0,2.357023,1.666667,-1.414214e-01,1.666667,3.333333,1.929306,-0.100000,2.000000,3.355194,0.424465,0.290015,1.0,3.062500,0.596212,0.500000,1.0,3.053802,0.087323,-0.174646,0.000000,3.333333,0.333333,-0.666667,0.000000,3.428571,2.935821,8.0,0.0,-0.666667,8.0,0.0
7,6.250000,4.234214,15.0,2.0,1.714286,15.0,2.0,3.232488,2.641892,9.428090e-01,2.500000,4.000000,2.420153,0.666667,2.500000,3.708851,0.802762,0.623380,1.5,3.900000,1.757840,1.250000,1.5,3.799702,1.057269,1.075189,0.500000,3.888889,0.831479,0.666667,0.500000,4.750000,4.621379,14.0,0.0,1.142857,14.0,0.0
8,5.777778,4.206476,15.0,2.0,-0.125000,15.0,2.0,4.065864,3.311910,1.010153e+00,3.200000,4.375000,2.471715,0.428571,3.000000,4.121485,1.178287,0.710941,2.0,4.125000,1.681703,0.600000,2.0,4.213748,1.163041,0.771586,1.000000,4.125000,0.828109,0.388889,1.000000,4.222222,4.603742,14.0,0.0,-0.750000,14.0,0.0
9,5.200000,4.366539,15.0,0.0,-0.333333,15.0,0.0,3.928371,3.146623,-5.551115e-17,3.833333,4.111111,2.446968,-0.250000,3.500000,4.421023,1.314672,0.598050,2.5,4.250000,1.586776,0.458333,2.5,4.364980,1.083336,0.457196,1.500000,4.400000,0.922557,0.458333,1.500000,4.200000,4.341019,14.0,0.0,-0.222222,14.0,0.0


In [188]:
def extract_features(df, segments, features):

    #main dataframe for storing stats
    X_tr = pd.DataFrame(index=range(segments), dtype=np.float64)
    
    for feature in features:
    
        #time series data
        x_full = pd.Series(df[f'{feature}'].values)
    
        for segment in range(segments):
            #take all of x until that point in time
            x = x_full[:segment+1]

            #compute stats on this feature
            X_tr.loc[segment, f'{feature}_mean'] = x.mean()
            X_tr.loc[segment, f'{feature}_std'] = x.std()
            X_tr.loc[segment, f'{feature}_max'] = x.max()
            X_tr.loc[segment, f'{feature}_min'] = x.min()

            X_tr.loc[segment, f'{feature}_mean_change_abs'] = np.mean(np.diff(x))
            X_tr.loc[segment, f'{feature}_abs_max'] = np.abs(x).max()
            X_tr.loc[segment, f'{feature}_abs_min'] = np.abs(x).min()

#             X_tr.loc[segment, 'max_to_min_diff'] = x.max() - np.abs(x.min())
#             X_tr.loc[segment, 'sum'] = x.sum()

#             X_tr.loc[segment, 'q95'] = np.quantile(x, 0.95)
#             X_tr.loc[segment, 'q99'] = np.quantile(x, 0.99)
#             X_tr.loc[segment, 'q05'] = np.quantile(x, 0.05)
#             X_tr.loc[segment, 'q01'] = np.quantile(x, 0.01)

#             X_tr.loc[segment, 'abs_q95'] = np.quantile(np.abs(x), 0.95)
#             X_tr.loc[segment, 'abs_q99'] = np.quantile(np.abs(x), 0.99)
#             X_tr.loc[segment, 'abs_q05'] = np.quantile(np.abs(x), 0.05)
#             X_tr.loc[segment, 'abs_q01'] = np.quantile(np.abs(x), 0.01)

#########################
        #     X_tr.loc[segment, 'trend'] = add_trend_feature(x)
        #     X_tr.loc[segment, 'abs_trend'] = add_trend_feature(x, abs_values=True)
        #     X_tr.loc[segment, 'abs_mean'] = np.abs(x).mean()
        #     X_tr.loc[segment, 'abs_std'] = np.abs(x).std()
#######################
#             X_tr.loc[segment, 'mad'] = x.mad()
#             X_tr.loc[segment, 'kurt'] = x.kurtosis()
#             X_tr.loc[segment, 'skew'] = x.skew()
#             X_tr.loc[segment, 'med'] = x.median()

            #rolling stats
#             for windows in [2, 4, 6, 8, 10, 12, 14, 16, 20, 25]:
            for windows in [2, 4, 6]:
                x_roll_std = x.rolling(windows).std().dropna().values
                x_roll_mean = x.rolling(windows).mean().dropna().values

                X_tr.loc[segment, 'ave_roll_std_' + str(windows)] = x_roll_std.mean()
                X_tr.loc[segment, 'std_roll_std_' + str(windows)] = x_roll_std.std()
                X_tr.loc[segment, 'av_change_abs_roll_std_' + str(windows)] = np.mean(np.diff(x_roll_std))
                X_tr.loc[segment, 'av_change_rate_roll_std_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_std) / x_roll_std[:-1]))[0])

                X_tr.loc[segment, 'ave_roll_mean_' + str(windows)] = x_roll_mean.mean()
                X_tr.loc[segment, 'std_roll_mean_' + str(windows)] = x_roll_mean.std()
                X_tr.loc[segment, 'av_change_abs_roll_mean_' + str(windows)] = np.mean(np.diff(x_roll_mean))
                X_tr.loc[segment, 'av_change_rate_roll_mean_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_mean) / x_roll_mean[:-1]))[0])
    X_tr = X_tr.fillna(value=0.0)
    return X_tr

## Model developement

In [ ]:
#XGBoost/CatBoost/LGBM

## Evaluation